In [ ]:
!wget --header="Host: doc-0o-0g-drive-data-export.googleusercontent.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9" --header="Cookie: AUTH_q9gllbu0nte95409rtp7q7s721mapf6v_nonce=vvim2copfnj4s" --header="Connection: keep-alive" "https://doc-0o-0g-drive-data-export.googleusercontent.com/download/b3n3f7ro91h5cr0rs39bilh5evv17sjb/768o5famgcgilpb6ac4qrt7scien22aj/1622371500000/5f250682-e6b5-4aab-827b-d896868e70dd/110819034128027560352/ADt3v-NxvzdHPn5Euk9c55oh3sWYpvKYmiz7B35xq5Vy82W1Z0HJ7U65UVPkDPhHQlCmiZcStHu-a5rbs01f59lF7sKTSTygXm6wdG57GTE8Y1AvemIZA2yQ9v0-k-shLui4w38Y2LxiK2b5KjEIfRKnes3HZ9UdxQQq8byqxQk52UAi7x_QR1gkn1Q_P1qHRyQ9-SDGf3GuCThgZFqApbJLm1EV4FAKw4YOZ4emfc454OMTsriV77CeKMCLw0jBLxFyNN42jR6nlUCbc1kdPaWKiyGV7L0WvJG674eiDORMMRabYJDkrhKJU4eGjCwd3WZqLu8nxG8Z?authuser=0&nonce=vvim2copfnj4s&user=110819034128027560352&hash=cqno48rsdfe6ilopvofn67rggi4vql4d" -c -O 'train-20210530T104616Z-001.zip'

# Import libraries

In [ ]:
import numpy as np 
import pandas as pd
import easyocr
import os
import pickle
import shutil
import re
import warnings
from zipfile import ZipFile
warnings.filterwarnings("ignore")

In [ ]:
import nltk 
nltk.download('words')
words = set(nltk.corpus.words.words())

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


In [ ]:
nltk.download('stopwords')
stopwords = set(nltk.corpus.stopwords.words('english')) 
stopwords.remove('not')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#load model for english language
reader = easyocr.Reader(['en'])

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


# Read text from all images with OCR

In [ ]:
def give_text_from_images(raw_path):# read text from images
    exception_data=[]
    dict_data={}                 # this dictionary contain image_name and text given by ocr
    root=os.listdir(raw_path)    #list all images in folder
    for image_name in root:
      path=raw_path+image_name
      try:
        output = reader.readtext(path)     # readtext method read text in images
        ocr_text=' '
        for i in output:      # output[0]=cordinates of text,output[1]=text ,output[2]=probability
          if i[2]>=0.25:       # select text which has probability greater than 0.25
            ocr_text+=i[1]+" "
        dict_data[image_name] =ocr_text
      except:                  # if images is not png, jpg in that case ignore it
        exception_data.append(image_name)
    return dict_data,exception_data    

# reading text from train folder images

In [ ]:
# reading text from train folder
train_text,train_exception=give_text_from_images('/content/drive/MyDrive/Meme/data/train/')

In [ ]:
# these images are in gif format  so I have removed these
train_exception

['fifxei.jpg',
 'fivqpe.jpg',
 'fminyf.jpg',
 'fn2aeh.jpg',
 'fk1zqf.jpg',
 'flzxlu.jpg',
 'fm0153.jpg']

In [ ]:
# save train_text in pickle file
pickle.dump(train_text, open("/content/drive/MyDrive/Meme/data/train_text", "wb"))
pickle.dump(train_exception, open("/content/drive/MyDrive/Meme/data/train_exception", "wb"))

# reading text from validation folder images

In [ ]:
validation_text,validation_exception=give_text_from_images('/content/drive/MyDrive/Meme/data/validation/')

In [ ]:
#these images are in gif format  so I have removed these
validation_exception

['fkm56e.jpg', 'fkp4hr.jpg', 'flq4z6.jpg', 'fiawku.jpg']

In [ ]:
#save validation_text in pickle file
pickle.dump(validation_text, open("/content/drive/MyDrive/Meme/data/validation_text", "wb"))
pickle.dump(validation_exception, open("/content/drive/MyDrive/Meme/data/validation_exception", "wb"))

# reading text from test folder images

In [ ]:
test_text,test_exception=give_text_from_images('/content/drive/MyDrive/Meme/data/test/')

In [ ]:
#save test_text in pickle file
pickle.dump(test_text, open("/content/drive/MyDrive/Meme/data/test_text", "wb"))
pickle.dump(test_exception, open("/content/drive/MyDrive/Meme/data/test_exception", "wb"))

In [ ]:
#these images are in gif format  so I have removed these
test_exception

['fn7b6i.jpg']

# seperate dank and not dank meme

In [ ]:
def fun(source,destination,data):
    # before this function create folder class_0 and class_1
    # for every class that we have in our  directory we check if it's dank_level=1 then we will move it in class_1
    # otherwise move in class_0
    # list_id for dank_level==1
    list_id=list(data[data.dank_level==1]['id'])
    if os.path.isdir(source):
        data_files = os.listdir(source)
        for file in data_files:
            file_temp=file.split(".")[0]
            if file_temp in list_id:
              #if dank_level=1 then we remove in class_1
              shutil.move(source+'/'+file,destination+'/class_1') 
            else:
              if file!=='class_1':
                shutil.move(source+'/'+file,destination+'/class_0')   
           

In [ ]:
#reading csv file of data
data=pd.read_csv('/content/drive/MyDrive/Meme/data/data.csv')

In [ ]:
#separating class 1 and 0
source_train = '/content/drive/MyDrive/Meme/data/train'
destination_train= '/content/drive/MyDrive/Meme/data/train'
# call function  to seperate class 1 and 0
fun(source_train,destination_train,data)

In [ ]:
source_validation = '/content/drive/MyDrive/Meme/data/validation'
destination_validation='/content/drive/MyDrive/Meme/data/validation'
#call function to seperate class 1 and 0
fun(source_validation,destination_validation,data)

# read saved ocr text from train and validation images

In [ ]:
train_text=pickle.load(open('/content/drive/MyDrive/Meme/data/train_text','rb'))
validation_text=pickle.load(open('/content/drive/MyDrive/Meme/data/validation_text','rb'))
test_text=pickle.load(open("/content/drive/MyDrive/Meme/data/test_text", "rb"))

In [ ]:
train_text=pd.DataFrame(train_text.items(), columns=['id','ocr_text'])
validation_text=pd.DataFrame(validation_text.items(), columns=['id','ocr_text'])
test_text=pd.DataFrame(test_text.items(), columns=['id','ocr_text'])

# steps for ocr text preprocessing

1. convert lower case
2. correct some cases  like this Don\'t,dont,dontt
2. remove some puctuation
3. remove word starts with @ and mem , words length<1 and words which are not  in english dictionary and stopwords
4. remove all non-words 

#some text features are
1. length of characters in ocr_text and no of words in ocr_text
2.length of characters in ocr_preprocess_text and no of words in ocr_preprocess_text

In [ ]:
def ocr_preprocess(text_data):
  for i,row in text_data.iterrows():
      value=row['ocr_text']
      value=value.strip()  #remove space at begining and ending
      text_data.loc[i,'ocr_text']=value   #from orignal text remove space
      value=value.lower()
      text_data.loc[i,'ocr_no_of_words']=len(value.split())    # count no_of_words before preprocessing 
      text_data.loc[i,'ocr_text_len']=len(value)                  #len of text
      
      # to handle  cases like this Don\'t,dont,dontt
      # replace Don\'t with do not use replace("n't", " not")
      # reference appliedai.com
      value=value.replace("won't", "will not").replace("cannot", "can not").replace("can't", "can not")\
                            .replace("n't", " not").replace("what's", "what is").replace("it's", "it is")\
                            .replace("'ve", " have").replace("i'm", "i am").replace("'re", " are")\
                            .replace("he's", "he is").replace("she's", "she is").replace("'ll", " will")\
                            .replace("'s", " is").replace("'ll", " will").replace("'m", " am")\
                            .replace("'t", " not").replace("'d", " would").replace("dont","do not ").replace("dont","do not")\
                            .replace("dontt","do not")
      # remove  \n,\t,(,),|,.,-,_,<,>,?,!,\,:,;,{,},*,#,%,
      value=re.sub(r'[\n\t()|.-_<>,?!\\:;{}*#%]+',' ', value)   
      # remove  word start with mem 
      # remove  word  start with @   like  @steve 
      # word less than len 1
      x=' '.join(x for x in value.split()  if   not x.startswith('@') and not x.startswith('mem') and  x in words and len(x)>1 and x not in stopwords )
      #remove puctuation from words  /,[,],_ like r/Minecraft,[BIDET,closeraoa_,ulone_slavic_boy, u/TheOriginalNav 
      #remove all digit
      x=re.sub("\W"," ",x)
      x=x.strip()
      text_data.loc[i,'ocr_preprocess']=x  
      text_data.loc[i,'ocr_preprocess_no_of_words']=len(x.split())
      text_data.loc[i,'ocr_preprocess_text_len']=len(x)
  return text_data


In [ ]:
#call preprocess function
validation_text=ocr_preprocess(validation_text)
train_text=ocr_preprocess(train_text)
test_text=ocr_preprocess(test_text)

In [ ]:
train_text.shape,validation_text.shape,test_text.shape

((3264, 7), (1852, 7), (1637, 7))

#seperate csv file for traing and validation

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/Meme/data/data.csv')
#drop duplicates from data
data=data.drop_duplicates(subset=['id'],keep='first').reset_index(drop=True)
data.shape

(67244, 18)

In [ ]:
train_text['id']=train_text['id'].apply(lambda x:x.split('.')[0])
train_data=train_text.merge(data,on='id',how='inner')
train_data=train_data.drop_duplicates(subset=['id'],keep='first').reset_index(drop=True)


In [ ]:
validation_text['id']=validation_text['id'].apply(lambda x:x.split('.')[0])
validation_data=validation_text.merge(data,on='id',how='inner')
validation_data=validation_data.drop_duplicates(subset=['id'],keep='first').reset_index(drop=True)


In [ ]:
test_text['id']=test_text['id'].apply(lambda x:x.split('.')[0])
test_data=test_text.merge(data,on='id',how='inner')
test_data=test_data.drop_duplicates(subset=['id'],keep='first').reset_index(drop=True)

In [ ]:
train_data.shape,validation_data.shape,test_data.shape

((3264, 24), (1615, 24), (1637, 24))

In [ ]:
def title_preprocess(text_data):
  for i,row in text_data.iterrows():
      value=row['title']
      value=value.strip()  #remove space at begining and ending
      text_data.loc[i,'title']=value   #from orignal text remove space
      value=value.lower()
      text_data.loc[i,'title_no_of_words']=len(value.split())    # count no_of_words before preprocessing 
      text_data.loc[i,'title_len']=len(value)                  #len of text
      
      # to handle  cases like this Don\'t,dont,dontt
      # replace Don\'t with do not use replace("n't", " not")
      # reference appliedai.com
      value=value.replace("won't", "will not").replace("cannot", "can not").replace("can't", "can not")\
                            .replace("n't", " not").replace("what's", "what is").replace("it's", "it is")\
                            .replace("'ve", " have").replace("i'm", "i am").replace("'re", " are")\
                            .replace("he's", "he is").replace("she's", "she is").replace("'ll", " will")\
                            .replace("'s", " is").replace("'ll", " will").replace("'m", " am")\
                            .replace("'t", " not").replace("'d", " would").replace("dont","do not ").replace("dont","do not")\
                            .replace("dontt","do not").replace("me_irl","me irl").replace("Me_irl",'me irl')
      # remove  \n,\t,(,),|,.,-,_,<,>,?,!,\,:,;,{,},*,#,%,
      value=re.sub(r'[\n\t()|.-_<>,?!\\:;{}*#%]+',' ', value)   
      # remove  word start with mem 
      # remove  word  start with @   like  @steve 
      # word less than len 1
      x=' '.join(x for x in value.split()  if   not x.startswith('@') and not x.startswith('mem') and len(x)>1 )
      #remove puctuation from words  /,[,],_ like r/Minecraft,[BIDET,closeraoa_,ulone_slavic_boy, u/TheOriginalNav 
      #remove all digit
      x=re.sub("\W"," ",x)
      x=x.strip()
      text_data.loc[i,'title_preprocess']=x  
      text_data.loc[i,'title_preprocess_no_of_words']=len(x.split())
      text_data.loc[i,'title_preprocess_text_len']=len(x)
  return text_data


In [ ]:
validation_data=title_preprocess(validation_data)
train_data=title_preprocess(train_data)
test_data=title_preprocess(test_data)

In [ ]:
#add_path of each images in csv file
def add_path(temp,data):
    for i,row in data.iterrows():
      root='/content/'+temp+'/'   #/content/validation/
      if row['dank_level']==1:
        data.loc[i,'path']=root+'class_1/'+row['id']+'.jpg'
      elif row['dank_level']==0:
        data.loc[i,'path']=root+'class_0/'+row['id']+'.jpg'
      else: 
        data.loc[i,'path'] ='error'  
    return data
train_data=add_path('train',train_data)
validation_data=add_path('validation',validation_data)    

In [ ]:
def test_data_path(data):
    for i,row in data.iterrows():
      root='/content/test/'
      data.loc[i,'path']=root+row['id']+'.jpg'
    return data
test_data=test_data_path(test_data)      


In [ ]:
train_data['text']= train_data['title_preprocess'].astype(str)+" "+train_data['ocr_preprocess'].astype(str)
validation_data['text']=validation_data['title_preprocess'].astype(str)+" "+validation_data['ocr_preprocess'].astype(str)
test_data['text']=test_data['title_preprocess'].astype(str)+' '+test_data['ocr_preprocess'].astype(str)

In [ ]:
train_data.head(2)

,id,ocr_text,ocr_no_of_words,ocr_text_len,ocr_preprocess,ocr_preprocess_no_of_words,ocr_preprocess_text_len,index,created_utc,downs,is_nsfw,media,subreddit,subscribers,thumbnail.height,thumbnail.thumbnail,thumbnail.width,title,ups,url,datetime_temp,time_of_day,ups_normed,dank_level,title_no_of_words,title_len,title_preprocess,title_preprocess_no_of_words,title_preprocess_text_len,path,text
0,fjn3gy,when acvedenalh bite their toungue The monster...,11.0,65.0,bite monster become,3.0,19.0,28606.0,1.584374e+09,0.0,False,https://i.redd.it/swm384fj32n41.jpg,r/memes,9552833.0,140.0,https://b.thumbs.redditmedia.com/C15Fr3fcu3dln...,140.0,Absolutely disgusting,975.0,/r/memes/comments/fjn3gy/absolutely_disgusting/,2020-03-16 10:51:43,2.0,1.020640e-04,1.0,2.0,21.0,absolutely disgusting,2.0,21.0,/content/train/class_1/fjn3gy.jpg,absolutely disgusting bite monster become
1,fjn514,42069 OUNLMATU PAHDOMORO 12 ;,5.0,29.0,,0.0,0.0,28575.0,1.584374e+09,0.0,False,https://i.redd.it/4hdh24op32n41.jpg,r/memes,9552833.0,105.0,default,140.0,Words cannot describe how much happiness I hav...,4.0,/r/memes/comments/fjn514/words_cannot_describe...,2020-03-16 10:54:27,2.0,4.187240e-07,0.0,9.0,57.0,words can not describe how much happiness have...,9.0,56.0,/content/train/class_0/fjn514.jpg,words can not describe how much happiness have...


In [ ]:
#save train_data and validation_data
train_data.to_csv('/content/drive/MyDrive/Meme/data/train_data.csv',index=False)
validation_data.to_csv('/content/drive/MyDrive/Meme/data/validation_data.csv',index=False)
test_data.to_csv('/content/drive/MyDrive/Meme/data/test_data.csv',index=False)

In [ ]:
print(train_data.shape)
print(validation_data.shape)
print(test_data.shape)

(3264, 31)
(1615, 31)
(1637, 31)
